In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import patsy as pts

from functools import partial
import re

In [2]:
pisadat = pd.read_csv("../data/pisadat_all.csv")

In [3]:
pisadat.columns

Index([u'CNT', u'GENDER', u'PRIMED', u'ESCS', u'FAMSTRUC', u'IMMIG', u'WEIGHT',
       u'CODE', u'SCHTYPE', u'SCHLOC', u'PVMATH', u'PVREAD', u'PVSCIE',
       u'SAMELNG', u'MATH_ISLP', u'READ_ISLP', u'SCIE_ISLP', u'ISLP_ANY',
       u'ISLP_ALL'],
      dtype='object')

In [4]:
variables = ['CNT', 'CODE', 'GENDER', 'PRIMED', 'FAMSTRUC',
            'IMMIG', 'WEIGHT', 'SCHTYPE', 'SCHLOC', 'SAMELNG', 'MATH_ISLP', 'ESCS']



Prepare data for analysis. Drop null values, add new variables ESCS_GR and DIFFLNG. 

In [5]:
data = pisadat[variables].dropna()
escs_quant = data.groupby("CNT").ESCS.agg(partial(np.percentile, q=25)).to_frame(name="ESCS_qunat").reset_index()
data = pd.merge(data, escs_quant, on="CNT")

data["ESCS_GR"] = "Adv"
data.loc[data.ESCS <= data.ESCS_qunat, "ESCS_GR"] = "Disadv"
data["DIFFLNG"] = 1 - data["SAMELNG"]

Recode categorical variables using specified reference levels, add interaction terms.

In [6]:
y, X = pts.dmatrices("MATH_ISLP ~ C(GENDER, Treatment(reference='Male')) + C(PRIMED, Treatment(reference='Yes')) + \
                    C(ESCS_GR, Treatment(reference='Adv')) + \
                  C(FAMSTRUC, Treatment(reference='Non-Single')) + C(IMMIG, Treatment(reference='Native')) + \
                  C(SCHTYPE, Treatment(reference='Private')) + C(SCHLOC, Treatment(reference='Urban') ) + \
                  C(DIFFLNG) + \
                  C(CNT):C(GENDER, Treatment(reference='Male')) +\
                  C(CNT):C(PRIMED, Treatment(reference='Yes')) +\
                  C(CNT):C(ESCS_GR, Treatment(reference='Adv')) +\
                  C(CNT):C(FAMSTRUC, Treatment(reference='Non-Single')) +\
                  C(CNT):C(IMMIG, Treatment(reference='Native')) +\
                  C(CNT):C(SCHTYPE, Treatment(reference='Private')) +\
                  C(CNT):C(SCHLOC, Treatment(reference='Urban') )+\
                  C(CNT):C(DIFFLNG)", data,
                    return_type="dataframe")
w = data["WEIGHT"]
X.drop("Intercept", axis=1, inplace=True)

In [7]:
X.shape

(400600, 566)

Fit weighted logistic regression with regularization, regularization parameter C is estimated via cross validation procedure.

In [49]:
clf = LogisticRegressionCV()
clf.fit(X, y, w)         

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [50]:
coef = pd.Series( clf.coef_[0], index=X.columns)
coef.shape

(566,)

Next match coefficients for each country. Note that for reference country (here Argentina) coefficients already characterize relationship, but for all other countries coefficients correspond to interaction terms and therefor show only the difference with reference level. 

In [51]:
countries_coef = pd.DataFrame(columns = ['CODE', 'CNT', 'GENDER', 'PRIMED',
            'ESCS_GR', 'FAMSTRUC', 'IMMIG', "SCHTYPE", "SCHLOC", "DIFFLNG"])
names_mtch = {
    "GENDER" : u"C(GENDER, Treatment(reference='Male'))[T.Female]",
    "PRIMED" : u"C(PRIMED, Treatment(reference='Yes'))[T.No]",
    "ESCS_GR": u"C(ESCS_GR, Treatment(reference='Adv'))[T.Disadv]",
    "FAMSTRUC": u"C(FAMSTRUC, Treatment(reference='Non-Single'))[T.Single]",
    "IMMIG": u"C(IMMIG, Treatment(reference='Native'))[T.Non-Native]",
    "SCHTYPE": u"C(SCHTYPE, Treatment(reference='Private'))[T.Public]",
    "SCHLOC": u"C(SCHLOC, Treatment(reference='Urban'))[T.Rural]",
    "DIFFLNG": u"C(DIFFLNG)[T.1]"
}

ref = "ARG" #Argentina
for code in data.CODE.unique():
    CNT = data[data.CODE == code]["CNT"].iloc[0]
    l = {
        "CODE": code,
        "CNT": CNT
    }
    for variable, name in names_mtch.items():
        feature_name = name
        if code != ref:
            feature_name = "C(CNT)[T.{}]:{}".format(CNT, name)
            
            if variable == "GENDER":
                feature_name1 = "C(CNT)[T.{}]:C(GENDER, Treatment(reference='Male'))[Female]".format(CNT)
                feature_name2 = "C(CNT)[T.{}]:C(GENDER, Treatment(reference='Male'))[Male]".format(CNT)
                l[variable] = coef[feature_name1] - coef[feature_name2]
                continue
        l[variable] = coef[feature_name]
    
        
    countries_coef = countries_coef.append(l, ignore_index=True)



In [52]:
countries_coef

,CODE,CNT,GENDER,PRIMED,ESCS_GR,FAMSTRUC,IMMIG,SCHTYPE,SCHLOC,DIFFLNG
0,ARE,United Arab Emirates,-0.082587,0.046454,0.031902,0.045512,-0.135141,0.128893,0.010312,-0.029956
1,ARG,Argentina,0.177342,0.605086,0.833532,0.350412,0.209203,0.367721,0.339496,0.249987
2,AUS,Australia,0.017855,-0.006972,-0.089877,-0.131823,-0.290387,-0.181829,-0.106224,-0.080815
3,AUT,Austria,0.058580,-0.011244,-0.084442,-0.092199,0.010984,-0.388724,-0.211579,0.045678
4,BEL,Belgium,-0.099980,-0.002861,-0.127681,-0.067677,0.029273,-0.059250,-0.180863,-0.077186
5,BGR,Bulgaria,-0.075450,0.009074,0.072535,-0.003968,0.004926,0.032173,0.082035,0.073186
6,BRA,Brazil,0.320266,-0.049617,-0.026037,-0.009682,0.135756,1.181128,-0.021256,-0.030466
7,CAN,Canada,-0.031089,-0.141824,-0.171080,-0.150258,-0.189295,-0.669784,-0.140617,-0.146416
8,CHE,Switzerland,-0.045375,0.005454,-0.129319,-0.089063,-0.027623,-0.496397,-0.288217,-0.344685
9,CHL,Chile,0.286022,0.038306,0.213076,0.074826,0.001577,0.362844,0.099062,0.024501


In order to get coefficients (which characterize relationship with the target variable) we sum values from the previous level with reference level (here Argentina).

In [53]:
countries_stat = countries_coef.copy()
ref_level = 1
for i in range(countries_coef.shape[0]):
    if i != ref_level:
        countries_stat.iloc[i,2:] = countries_coef.iloc[i,2:] + countries_coef.iloc[ref_level,2:]

In [54]:
countries_stat

,CODE,CNT,GENDER,PRIMED,ESCS_GR,FAMSTRUC,IMMIG,SCHTYPE,SCHLOC,DIFFLNG
0,ARE,United Arab Emirates,0.094755,0.651540,0.865433,0.395923,0.074062,0.496614,0.349808,0.220031
1,ARG,Argentina,0.177342,0.605086,0.833532,0.350412,0.209203,0.367721,0.339496,0.249987
2,AUS,Australia,0.195197,0.598114,0.743654,0.218589,-0.081184,0.185893,0.233272,0.169172
3,AUT,Austria,0.235923,0.593842,0.749090,0.258212,0.220187,-0.021003,0.127917,0.295665
4,BEL,Belgium,0.077362,0.602226,0.705851,0.282734,0.238476,0.308471,0.158634,0.172801
5,BGR,Bulgaria,0.101892,0.614160,0.906067,0.346444,0.214129,0.399894,0.421531,0.323173
6,BRA,Brazil,0.497609,0.555469,0.807495,0.340730,0.344958,1.548850,0.318240,0.219521
7,CAN,Canada,0.146253,0.463262,0.662452,0.200153,0.019908,-0.302063,0.198880,0.103571
8,CHE,Switzerland,0.131967,0.610540,0.704213,0.261349,0.181580,-0.128676,0.051279,-0.094698
9,CHL,Chile,0.463365,0.643392,1.046608,0.425238,0.210779,0.730565,0.438558,0.274488


In [55]:
countries_stat.to_csv("countries_stat_longreg.csv", index=False)